# Fall Detection Project

## 3D Human Pose Estimation

Using **[VideoPose3D](https://github.com/facebookresearch/VideoPose3D)** by FB AI Research. 

Leveraging the model and weights provided by FAIR and following the [Inference in the wild](https://github.com/facebookresearch/VideoPose3D/blob/master/INFERENCE.md) tutorial. 

The following steps uses [Detectron](https://github.com/facebookresearch/Detectron) to infer 2D keypoints from videos, convert them to a custom dataset, and infer 3D poses. 


### 1. Setup
#### Install Pytorch & Caffe2

In [0]:
!wget https://anaconda.org/pytorch/pytorch/1.2.0/download/linux-64/pytorch-1.2.0-py3.6_cuda10.0.130_cudnn7.6.2_0.tar.bz2
!tar xvjf pytorch-1.2.0-py3.6_cuda10.0.130_cudnn7.6.2_0.tar.bz2
!cp -r lib/python3.6/site-packages/* /usr/local/lib/python3.6/dist-packages/

Check if Caffe2 was built

In [2]:
# To check if Caffe2 build was successful
!python -c 'from caffe2.python import core' 2>/dev/null && echo "Success" || echo "Failure"

# To check if Caffe2 GPU build was successful
!python -c 'from caffe2.python import workspace; print(workspace.NumCudaDevices())'

Success
1


#### Install COCO API 

In [0]:
!apt-get install python-dev
!pip install cython
!pip install pycocotools
!git clone https://github.com/cocodataset/cocoapi.git
!cd cocoapi/PythonAPI && make install

import os
os.environ['COCOAPI'] = ":/content/cocoapi"

#### Setup Detectron

In [0]:
!git clone https://github.com/facebookresearch/detectron
!pip install -r detectron/requirements.txt
!cd detectron && make


In [0]:
# Run test:
!python detectron/detectron/tests/test_spatial_narrow_as_op.py

#### Setup VideoPose3D

In [0]:
# Install VideoPose3D
!git clone https://github.com/facebookresearch/VideoPose3D

# Copy infer_video.py to Detectron folder
!cp VideoPose3D/inference/infer_video.py detectron/tools/infer_video.py

#### Download the pretrained model for generating 3D predictions
This model expects input keypoints in COCO format (generated by the pretrained Detectron model) and outputs 3D joint positions in Human3.6M format. 

In [6]:
#  We put this model in the checkpoint directory of this repo.
!mkdir VideoPose3D/checkpoint
os.chdir('VideoPose3D/checkpoint')

# Download Human3.6M weights
!wget https://dl.fbaipublicfiles.com/video-pose-3d/pretrained_h36m_detectron_coco.bin
os.chdir('../..')

--2019-09-09 04:20:03--  https://dl.fbaipublicfiles.com/video-pose-3d/pretrained_h36m_detectron_coco.bin
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.20.6.166, 104.20.22.166, 2606:4700:10::6814:6a6, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.20.6.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67892577 (65M) [application/octet-stream]
Saving to: ‘pretrained_h36m_detectron_coco.bin’

pretrained_h36m_det 100%[===================>]  64.75M  80.5MB/s    in 0.8s    

2019-09-09 04:20:04 (80.5 MB/s) - ‘pretrained_h36m_detectron_coco.bin’ saved [67892577/67892577]



## 2. Video Processing

#### Download video from Youtube. 

*Note: The script assumes that a video depicts exactly one person. In case of multiple people visible at once, the script will select the person corresponding to the bounding box with the highest confidence, which may cause glitches.*


Download a Youtube Video for 3D Pose Estimation. 
Necessary to specify Youtube_id:

In [0]:
!pip install -q youtube-dl

YOUTUBE_ID = 'WV-Sf5-aCcc'  # e.g.: Capoeira 

# download video
!youtube-dl -f 'bestvideo[ext=mp4]' --output "youtube_video.%(ext)s" https://www.youtube.com/watch?v=$YOUTUBE_ID

#### Extract a clip

Since the script expects a single-person scenario, you may want to extract a portion of your video. Here we use ffmpeg library. 

e.g.:
```
ffmpeg -i input.mp4 -ss 1:00 -to 1:30 -c copy output.mp4

```
extracts a clip from minute 1:00 to minute 1:30 of input.mp4, and exports it to output.mp4.


In [0]:
!mkdir videos   

# Extract a 10sec clip from minute 1:05
!ffmpeg -y -loglevel info -i youtube_video.mp4 -ss 00:01:05 -t 00:00:10 videos/input_video.mp4




Optionally, you can also adapt the frame rate of the video. Most videos have a frame rate of about 25 FPS, but our Human3.6M model was trained on 50-FPS videos. Since our model is robust to alterations in speed, this step is not very important and can be skipped, but if you want the best possible results you can use ffmpeg again for this task:
```
ffmpeg -i input.mp4 -filter "minterpolate='fps=50'" -crf 0 output.mp4
```

In [0]:
#!ffmpeg -i videos/video.mp4 -filter "minterpolate='fps=50'" -crf 0 videos/input_video50fps.mp4

## 3. Inferring 2D keypoints with Detectron

To infer keypoints from all the mp4 videos in our  **input directory** (`/videos`), run the following. 

The results will be exported to our **output directory** (`/output`) directory as custom NumPy archives (`.npz` files). You can change the video extension in `--image-ext` (ffmpeg supports a wide range of formats).

In [0]:
!mkdir output
!python detectron/tools/infer_video.py \
    --cfg detectron/configs/12_2017_baselines/e2e_keypoint_rcnn_R-101-FPN_s1x.yaml \
    --output-dir output \
    --image-ext mp4 \
    --wts https://dl.fbaipublicfiles.com/detectron/37698009/12_2017_baselines/e2e_keypoint_rcnn_R-101-FPN_s1x.yaml.08_45_57.YkrJgP6O/output/train/keypoints_coco_2014_train:keypoints_coco_2014_valminusminival/generalized_rcnn/model_final.pkl \
   videos
    
#	  --wts https://dl.fbaipublicfiles.com/detectron/37698009/12_2017_baselines/e2e_keypoint_rcnn_R-101-FPN_s1x.yaml.08_45_57.YkrJgP6O/output/train/keypoints_coco_2014_train:keypoints_coco_2014_valminusminival/generalized_rcnn/model_final.pkl \

    

## 4. Creating a Custom Dataset
Here we prepare Detectron Output to fit VideoPose3D Input.

The following creates a custom dataset named `myvideos` (which contains all the videos our in output directory (`/detectron_output`), each of which is mapped to a different subject and saved to `data_2d_custom_myvideos.npz`).

In [0]:
!mkdir /content/VideoPose3D/data/detectron_output
!cp /content/output/video.mp4.npz /content/VideoPose3D/data/detectron_output/video.mp4.npz 
os.chdir('VideoPose3D/data') # This script must be launched from the "data" directory
!python prepare_data_2d_custom.py -i detectron_output -o myvideos
os.chdir('../../')

## 5. rendering a custom video and exporting coordinates.

Finally use use the visualization feature to render a video of the 3D joint predictions. You must specify the `custom` dataset (`-d custom`), the input keypoints as exported in the previous step (`-k myvideos`), the correct `architecture/checkpoint`, and the action custom (`--viz-action custom`). The subject is the file name of the input video, and the camera is always `0`.


You can also export the 3D joint positions (in camera space) to a NumPy archive. To this end, replace `--viz-output` with `--viz-export` and specify the file name.

In [0]:
!cp /content/videos/video.mp4 /content/VideoPose3D/video.mp4
os.chdir('VideoPose3D')

!python run.py -d custom -k myvideos -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject video.mp4 --viz-action custom --viz-camera 0 --viz-video video.mp4 --viz-output output.mp4 --viz-export outputfile --viz-size 6

#working version 
#!python run.py -k gt -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_cpn.bin --viz-export My3dDataExport --viz-output output.mp4

## 6. Display Results 

#### Joint Export 

In [0]:
#inspect joints export 

import numpy as np
data  = np.load('outputfile.npy')
lst = data
for item in lst[:10]:
    print(item)
    


#### Joint Video

In [25]:
from moviepy.editor import *
clip = VideoFileClip('/content/VideoPose3D/output.mp4').subclip((0,01.00),(0,09.00) )
clip.ipython_display(width=960, height=720) 

100%|█████████▉| 240/241 [00:03<00:00, 72.25it/s]


In [0]:
#display video (Option #2)
# def show_local_mp4_video(file_name, width=640, height=480):
#   import io
  
#   import base64
#   from IPython.display import HTML
#   video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
#   return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
#                         <source src="data:video/mp4;base64,{2}" type="video/mp4" />
#                       </video>'''.format(width, height, video_encoded.decode('ascii')))

# show_local_mp4_video('output.mp4', width=960, height=720)

#### Download Joint Export & Video

In [0]:
from google.colab import files


files.download('output.mp4')
files.download('outputfile.npy')